In [1]:
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

import tensorflow as tf


from itertools import chain
from process_data import load_entities, load_task, vectorize, vectorize_kv_pairs, save_pickle, load_pickle, load_kv_pairs, lower_list


Using TensorFlow backend.


In [3]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

def load_task(fpath, is_babi, max_length=None):
    with open (fpath, encoding='utf-8') as f:
        lines = f.readlines()
        data, story = [], []
        for l in lines:
            l = l.rstrip()
            turn, left = l.split(' ', 1)
            
            if turn == '1': # new story
                story = []

            if '\t' in left: # question
                q, a = left.split('\t', 1)
                q = tokenize(q)
                q = lower_list(q)
                if q[-1] == '?':
                    q = q[:-1]
                if '\t' in a:
                    a = a.split('\t')[0] # discard reward
                a = a.split('|') # may contain several labels
                a = lower_list(a)

                substory = [x for x in story if x]

                data.append((substory, q, a))
                story.append('')
            else: # normal sentence
                s = tokenize(left)
                if s[-1] == '.':
                    s = s[:-1]
                s = lower_list(s)
                story.append(s)

    if is_babi:
        flatten = lambda data: reduce(lambda x, y: x + y, data)
        data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
        
    return data

def vectorize(data, w2i, story_maxlen, query_maxlen, entities=None):
    if entities:
        e2i = dict((e, i) for i, e in enumerate(entities))

    S, Q, A = [], [], []
    for story, question, answer in data:
        # Vectroize story
        s_pad_len = max(0, story_maxlen - len(story))
        s = [w2i[w] for w in story] + [0] * s_pad_len

        # Vectroize question
        q_pad_len = max(0, query_maxlen - len(question))
        q = [w2i[w] for w in question] + [0] * q_pad_len
        q = q[:query_maxlen]

        # Vectroize answer
        if entities:
            y = np.zeros(len(entities), dtype='byte')
            for a in answer:
                y[e2i[a]] = 1
        else:
            y = np.zeros(len(w2i) + 1) # +1 for nil word
            for a in answer:
                y[w2i[a]] = 1

        S.append(s)
        Q.append(q)
        A.append(y)
    
    S = np.array(S, dtype=np.uint16)
    Q = np.array(Q, dtype=np.uint16)
    A = np.array(A, dtype='byte')

    return S, Q, A

def vectorize_kv_pairs(kv_pairs, memory_size, entities):
    vec_kv_pairs = []
    w2i = dict((e, i) for i, e in enumerate(entities))
    w2i['directed_by'] = len(w2i)
    w2i['written_by'] = len(w2i)
    w2i['starred_actors'] = len(w2i)
    w2i['release_year'] = len(w2i)
    w2i['has_genre'] = len(w2i)
    w2i['has_tags'] = len(w2i)
    w2i['has_plot'] = len(w2i)
    for ent_list in kv_pairs:
#         print('----ent_list', ent_list)
#         print(len(ent_list))
        kv = [w2i[e] for e in ent_list if e in w2i]
        mem_pad_len = max(0, memory_size - len(kv))
        vec_kv_pairs.append(kv + [0] * mem_pad_len)


    return np.array(vec_kv_pairs, dtype=np.uint16)

In [4]:
is_babi = True
if is_babi:
    train_stories = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_train.txt', is_babi)
    test_stories = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_test.txt', is_babi)
else:
    N = 5000
    train_stories = load_pickle('mov_task1_qa_pipe_train.pickle')[:N]
    test_stories = load_pickle('mov_task1_qa_pipe_test.pickle')[:N]
    kv_pairs = load_pickle('mov_kv_pairs.pickle')
    train_kv_indices = load_pickle('mov_train_kv_indices.pickle')[:N]
    test_kv_indices = load_pickle('mov_test_kv_indices.pickle')[:N]
    print(len(train_kv_indices))
#     flatten = lambda data: reduce(lambda x, y: x + y, data)
    train_kv = [ [kv_pairs[ind] for ind in indices] for indices in train_kv_indices ]
    test_kv = [ [kv_pairs[ind] for ind in indices] for indices in test_kv_indices ]
    train_kv = np.array([list(chain(*train_kv[x])) for x in range(len(train_kv))])
#     print('before flatten train_kv[0]', train_kv[0])
#     train_kv = np.array(list(chain.from_iterable(train_kv)))
#     train_kv = list(chain.from_iterable(train_kv))
#     test_kv = np.array(list(chain.from_iterable(test_kv)))
#     print('after flatten train_kv[0]', train_kv[0])
#     train_kv = np.array(train_kv)
    print(len(train_kv), train_kv[0])
    
    entities = load_pickle('mov_entities.pickle')
    entity_size = len(entities)

vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + answer)
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

w2i = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize(train_stories,
                                                               w2i,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize(test_stories,
                                                            w2i,
                                                            story_maxlen,
                                                            query_maxlen)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)


-
Vocab size: 40 unique words
Story max length: 525 words
Query max length: 7 words
Number of training stories: 1000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['bill', 'travelled', 'to', 'the', 'office', 'bill', 'picked', 'up', 'the', 'football', 'there', 'bill', 'went', 'to', 'the', 'bedroom', 'bill', 'gave', 'the', 'football', 'to', 'fred'], ['what', 'did', 'bill', 'give', 'to', 'fred'], ['football'])
-
Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (1000, 525)
inputs_test shape: (1000, 525)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (1000, 7)
queries_test shape: (1000, 7)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (1000, 40)
answers_test shape: (1000, 40)


/home/jonki/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [ ]:
print('train_kv[0]:', train_kv[0], ', mem_size:', len(train_kv[0]))
# mem_maxlen = max(map(len, (x for x in train_kv+test_kv)))
train_mem_maxlen = max(map(len, (x for x in train_kv)))
test_mem_maxlen = max(map(len, (x for x in test_kv)))
mem_maxlen = max(train_mem_maxlen, test_mem_maxlen)

print('mem_maxlen:', mem_maxlen)
vec_train_kv = vectorize_kv_pairs(train_kv, mem_maxlen, vocab)

In [ ]:
print(train_kv[0])
print(vec_train_kv[0])

In [ ]:
def MemNNKV(mem_size, query_maxlen, vocab_size, entity_size, embd_size):
    print('mem_size:', mem_size)
    print('q_max', query_maxlen)
    print('embd_size', embd_size)
    print('vocab_size', vocab_size)
    print('entity_size', entity_size)
    # placeholders
    key = Input((mem_size,), name='Key_Input')
    val = Input((mem_size,), name='Val_Input')
    question = Input((query_maxlen,), name='Question_Input')

    # encoders
    # memory encoders
    key_encoder = Sequential()
    key_encoder.add(Embedding(input_dim=vocab_size, output_dim=embd_size, input_length=mem_size))
    val_encoder = Sequential()
    val_encoder.add(Embedding(input_dim=vocab_size, output_dim=embd_size, input_length=mem_size))
    # output: (samples, mem_size, embd_size)

    # embed the question into a sequence of vectors
    question_encoder = Sequential()
    question_encoder.add(Embedding(input_dim=vocab_size,
                                   output_dim=embd_size,
                                   input_length=query_maxlen))
    question_encoder.add(Dropout(0.3))
    # output: (samples, query_maxlen, embd_size)

    # encode input sequence and questions (which are indices)
    # to sequences of dense vectors
    key_encoded = key_encoder(key)
    val_encoded = val_encoder(val)
    question_encoded = question_encoder(question) # (None, query_max_len, embd_size)
    print('--encoded..')
    print('key_encoded', key_encoded.shape)
    print('que_encoded', question_encoded.shape)

#     # add the match matrix with the second input vector sequence
# #     response = add([match, input_encoded_c])  # (samples, mem_size, query_maxlen)
#     response = add([match, val_encoded])  # (samples, mem_size, query_maxlen)
#     response = Permute((2, 1))(response)  # (samples, query_maxlen, mem_size)

#     # concatenate the match matrix with the question vector sequence
#     answer = concatenate([response, question_encoded])
  
    ph = dot([question_encoded, key_encoded], axes=(2, 2))
    ph = Permute((2, 1))(ph)
    print('----ph', ph.shape)
    print('---val', val_encoded.shape)
    o = dot([ph, val_encoded], axes=(1, 1))
    print('o.shape', o.shape)
    R = Dense(embd_size, input_shape=(embd_size,))
    
#     q_o = question_encoded + o
#     print('q+o', q_o.shape)
#     print('R', R.shape)
    q2 = R(question_encoded + o)
    print('q2', q2.shape)
    
    cand_encoder = Sequential()
    cand_encoder.add(Embedding(input_dim=entity_size, output_dim=embd_size))#, input_length=1))
#     cand_encoder.add(Dropout(0.3))
#     y_data = np.zeros((entity_size, entity_size), dtype=np.uint8)
#     for i in range(entity_size):
#         y_data[i][i] = 1
    y_data = tf.diag(tf.ones(shape=[entity_size]))
    print('y_data', y_data.shape)
    y_encoded = cand_encoder(y_data)
    print('y_encoded', y_encoded.shape)
#     answer = dot([q2, y_encoded], axes=(2, 2))
    answer = 
    print('answer', answer.shape)
#     print('answer.shape', answer.shape)
    
    # the original paper uses a matrix multiplication for this reduction step.
    # we choose to use a RNN instead.
#     answer = LSTM(32)(answer)  # (samples, 32)
    # one regularization layer -- more would probably be needed.
    
#     answer = Dropout(0.3)(answer)
    answer = Dense(entity_size)(answer)  # (samples, vocab_size)
    # we output a probability distribution over the vocabulary
    answer = Activation('softmax')(answer)

    # build the final model
    model = Model([key, val, question], answer)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

embd_size = 128
memnn_kv = MemNNKV(mem_maxlen, query_maxlen, vocab_size, entity_size, embd_size)
print('---input_train', inputs_train.shape)
memnn_kv.fit([vec_train_kv, vec_train_kv, queries_train], answers_train,
          batch_size=32,
          epochs=10)
#           validation_data=([inputs_train, inputs_train, queries_test], answers_test))

In [7]:
def MemNN(story_maxlen, query_maxlen, vocab_size, embd_size):
    # placeholders
    input_sequence = Input((story_maxlen,))
    question = Input((query_maxlen,))

    # encoders
    # embed the input sequence into a sequence of vectors
    input_encoder_m = Sequential()
    input_encoder_m.add(Embedding(input_dim=vocab_size,
                                  output_dim=embd_size))
    input_encoder_m.add(Dropout(0.3))
    # output: (samples, story_maxlen, embedding_dim)

    # embed the input into a sequence of vectors of size query_maxlen
    input_encoder_c = Sequential()
    input_encoder_c.add(Embedding(input_dim=vocab_size,
                                  output_dim=query_maxlen))
    input_encoder_c.add(Dropout(0.3))
    # output: (samples, story_maxlen, query_maxlen)

    # embed the question into a sequence of vectors
    question_encoder = Sequential()
    question_encoder.add(Embedding(input_dim=vocab_size,
                                   output_dim=embd_size,
                                   input_length=query_maxlen))
    question_encoder.add(Dropout(0.3))
    # output: (samples, query_maxlen, embedding_dim)

    # encode input sequence and questions (which are indices)
    # to sequences of dense vectors
    input_encoded_m = input_encoder_m(input_sequence)
    input_encoded_c = input_encoder_c(input_sequence)
    question_encoded = question_encoder(question) # (None, query_max_len, embd_size)

    # compute a 'match' between the first input vector sequence
    # and the question vector sequence
    # shape: `(samples, story_maxlen, query_maxlen)`
    match = dot([input_encoded_m, question_encoded], axes=(2, 2))
    match = Activation('softmax')(match) # (None, max_storylen, query_maxlen)

    # add the match matrix with the second input vector sequence
    response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
    response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)
    print('---')
    print('match',match.shape)
    print('input_c', input_encoded_c.shape)
    print('response', response.shape)
    # concatenate the match matrix with the question vector sequence
    answer = concatenate([response, question_encoded])
    print('---')
    print('resp.shape', response.shape)
    print('q_enc.shape', question_encoded.shape)
    print('answer.shape', answer.shape)

    # the original paper uses a matrix multiplication for this reduction step.
    # we choose to use a RNN instead.
    answer = LSTM(32)(answer)  # (samples, 32)

    # one regularization layer -- more would probably be needed.
    answer = Dropout(0.3)(answer)
    answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
    # we output a probability distribution over the vocabulary
    answer = Activation('softmax')(answer)

    # build the final model
    model = Model([input_sequence, question], answer)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

embd_size = 64
model = MemNN(story_maxlen, query_maxlen, vocab_size, embd_size)

# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=60,
          validation_data=([inputs_test, queries_test], answers_test))

---
match (?, 525, 7)
input_c (?, 525, 7)
response (?, 7, 525)
---
resp.shape (?, 7, 525)
q_enc.shape (?, 7, 64)
answer.shape (?, 7, 589)
Train on 1000 samples, validate on 1000 samples
Epoch 1/60
1000/1000 [==============================] - 0s - loss: 2.7039 - acc: 0.1420 - val_loss: 2.2099 - val_acc: 0.1370
Epoch 2/60
1000/1000 [==============================] - 0s - loss: 2.2596 - acc: 0.1550 - val_loss: 1.9926 - val_acc: 0.1950
Epoch 3/60
1000/1000 [==============================] - 0s - loss: 1.9480 - acc: 0.2310 - val_loss: 1.6834 - val_acc: 0.2620
Epoch 4/60
1000/1000 [==============================] - 0s - loss: 1.6932 - acc: 0.2850 - val_loss: 1.5089 - val_acc: 0.3010
Epoch 5/60
1000/1000 [==============================] - 0s - loss: 1.5561 - acc: 0.2820 - val_loss: 1.4359 - val_acc: 0.2830
Epoch 6/60
1000/1000 [==============================] - 0s - loss: 1.4696 - acc: 0.3400 - val_loss: 1.4126 - val_acc: 0.2840
Epoch 7/60
1000/1000 [==============================] - 0s - los

In [ ]:
def MemNNKV_bk(mem_size, query_maxlen, vocab_size, embd_size):
    print('mem_size:', mem_size)
    print('q_max', query_maxlen)
    print('embd_size', embd_size)
    print('vocab_size', vocab_size)
    # placeholders
    key = Input((mem_size,))
    val = Input((mem_size,))
    question = Input((query_maxlen,))

    # encoders
    # memory encoders
    key_encoder = Sequential()
    key_encoder.add(Embedding(input_dim=vocab_size, output_dim=embd_size))#, input_length=query_maxlen))
    val_encoder = Sequential()
    val_encoder.add(Embedding(input_dim=vocab_size, output_dim=query_maxlen))#, input_length=query_maxlen))
    # output: (samples, mem_size, embd_size)

    # embed the question into a sequence of vectors
    question_encoder = Sequential()
    question_encoder.add(Embedding(input_dim=vocab_size,
                                   output_dim=embd_size,
                                   input_length=query_maxlen))
    question_encoder.add(Dropout(0.3))
    # output: (samples, query_maxlen, embd_size)

    # encode input sequence and questions (which are indices)
    # to sequences of dense vectors
    key_encoded = key_encoder(key)
    val_encoded = val_encoder(val)
    question_encoded = question_encoder(question) # (None, query_max_len, embd_size)

    # compute a 'match' between the first input vector sequence
    # and the question vector sequence
    # shape: `(samples, mem_size, query_maxlen)`
#     match = dot([input_encoded_m, question_encoded], axes=(2, 2))
    match = dot([key_encoded, question_encoded], axes=(2, 2))
    match = Activation('softmax')(match) # (None, mem_size, query_maxlen)
    print('match.shape', match.shape)

    # add the match matrix with the second input vector sequence
#     response = add([match, input_encoded_c])  # (samples, mem_size, query_maxlen)
    response = add([match, val_encoded])  # (samples, mem_size, query_maxlen)
    response = Permute((2, 1))(response)  # (samples, query_maxlen, mem_size)

    # concatenate the match matrix with the question vector sequence
    answer = concatenate([response, question_encoded])
    print('---')
    print('resp.shape', response.shape)
    print('q_enc.shape', question_encoded.shape)
    print('answer.shape', answer.shape)

    # the original paper uses a matrix multiplication for this reduction step.
    # we choose to use a RNN instead.
    answer = LSTM(32)(answer)  # (samples, 32)

    # one regularization layer -- more would probably be needed.
    answer = Dropout(0.3)(answer)
    answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
    # we output a probability distribution over the vocabulary
    answer = Activation('softmax')(answer)

    # build the final model
    model = Model([key, val, question], answer)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# embd_size = 64
# memnn_kv = MemNNKV(story_maxlen, query_maxlen, vocab_size, embd_size)
# print('---input_train', inputs_train.shape)
# memnn_kv.fit([inputs_train, inputs_train, queries_train], answers_train,
#           batch_size=32,
#           epochs=10,
#           validation_data=([inputs_train, inputs_train, queries_test], answers_test))